In [93]:
import collections
import math
import os

import docker
import fire
import torch

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, AutoModelForCausalLM
import time
import evaluate
import pandas as pd
import numpy as np

import peft
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig

import transformers
from datasets import Dataset



from fedn.utils.helpers.helpers import get_helper, save_metadata, save_metrics 


In [51]:


def save_model_custom(model, path):
    # Save configuration
    model.config.to_json_file(path + '.json')

    # Save weights in NumPy format
    weights_dict = {k: v.float().cpu().numpy() for k, v in model.state_dict().items()}
    np.savez_compressed(path, **weights_dict)


model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

save_model_custom(model, "/Users/nash/Project/fedn/fedn/examples/mnist-pytorch/test/test")


In [128]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration(config)


In [129]:
model.generate()

/Users/nash/Project/fedn/lib/python3.9/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[   0, 2198, 2198, 2198, 2198, 2198, 2198, 2198, 2198, 2198, 2198, 2198,
         2198, 2198, 2198, 2198, 2198, 2198, 2198, 2198]])

In [88]:
PATH = '/Users/nash/Downloads/one_clients_one_round'

In [89]:
a = np.load(PATH)

In [90]:
weights = []
for key in (a.files):
    weights.append(a[str(key)])

In [91]:
params_dict = zip(model.state_dict().keys(), weights)
state_dict = collections.OrderedDict({key: torch.tensor(x) for key, x in params_dict})
xxxx = model.load_state_dict(state_dict, strict=True)

RuntimeError: Error(s) in loading state_dict for T5ForConditionalGeneration:
	size mismatch for encoder.block.0.layer.0.SelfAttention.k.weight: copying a param with shape torch.Size([32, 768]) from checkpoint, the shape in current model is torch.Size([768, 768]).
	size mismatch for encoder.block.0.layer.0.SelfAttention.v.weight: copying a param with shape torch.Size([768, 32]) from checkpoint, the shape in current model is torch.Size([768, 768]).
	size mismatch for encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: copying a param with shape torch.Size([768, 768]) from checkpoint, the shape in current model is torch.Size([32, 12]).
	size mismatch for encoder.block.0.layer.0.layer_norm.weight: copying a param with shape torch.Size([32, 768]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for encoder.block.0.layer.1.DenseReluDense.wi_0.weight: copying a param with shape torch.Size([768, 32]) from checkpoint, the shape in current model 

In [61]:
type(xxxx)

torch.nn.modules.module._IncompatibleKeys

In [64]:
dataset = pd.read_csv("/Users/nash/Downloads/test.csv")
dialogues = dataset['dialogue'][0:10]
human_baseline_summaries = dataset['summary'][0:10]

In [72]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
    
    Summarize the following conversation.

    {dialogue}

    Summary: """
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    peft_model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)   
    peft_model_summaries.append(peft_model_text_output)      

In [74]:
len(peft_model_summaries)

10

In [75]:
rouge = evaluate.load('rouge')
    
peft_model_results = rouge.compute(
        predictions=peft_model_summaries,
        references=human_baseline_summaries[0:len(peft_model_summaries)],
        use_aggregator=True,
        use_stemmer=True,
)

In [76]:
peft_model_results

{'rouge1': 0.24010841010841008,
 'rouge2': 0.06962962962962962,
 'rougeL': 0.19956481862364217,
 'rougeLsum': 0.19984027301674356}

In [92]:
import torch
import torch.nn as nn
from transformers import T5Model, T5Config

class MyT5Model(nn.Module):
    def __init__(self, config):
        super(MyT5Model, self).__init__()
        self.t5 = T5Model(config)

    def forward(self, input_ids, attention_mask=None):
        # Pass input through the T5 model
        outputs = self.t5(input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state

# Create a T5 configuration based on the provided settings
t5_config = T5Config(
    vocab_size=32128,
    d_model=768,
    d_ff=2048,
    num_layers=12,
    num_heads=12,
    d_kv=64,
    dropout_rate=0.1,
    layer_norm_epsilon=1e-6,
    is_encoder_decoder=True,
    output_past=True,
    relative_attention_max_distance=128,
    relative_attention_num_buckets=32,
    task_specific_params={
        "summarization": {
            "early_stopping": True,
            "length_penalty": 2.0,
            "max_length": 200,
            "min_length": 30,
            "no_repeat_ngram_size": 3,
            "num_beams": 4,
            "prefix": "summarize: "
        },
        "translation_en_to_de": {
            "early_stopping": True,
            "max_length": 300,
            "num_beams": 4,
            "prefix": "translate English to German: "
        },
        # Add other task-specific parameters here
    }
)

# Instantiate your custom T5 model
my_t5_model = MyT5Model(t5_config)

# Example usage:
input_ids = torch.tensor([[1, 2, 3, 4]])  # Replace with your input data
attention_mask = torch.tensor([[1, 1, 1, 1]])  # Replace with attention mask
outputs = my_t5_model(input_ids, attention_mask=attention_mask)
print(outputs.shape)  # Shape of the output hidden states


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds